## Import relevant libraries

In [1]:
import numpy as np
import tensorflow as tf

import tensorflow_datasets as tfds

## Data

In [2]:
mnist_dataset, mnist_info = tfds.load(name='mnist', with_info=True, as_supervised=True)

mnist_train, mnist_test = mnist_dataset['train'], mnist_dataset['test']

num_validation_samples = 0.1 * mnist_info.splits['train'].num_examples
num_validation_samples = tf.cast(num_validation_samples, tf.int64)

num_test_samples = mnist_info.splits['test'].num_examples
num_test_samples = tf.cast(num_test_samples, tf.int64)


def scale(image, label):
    image = tf.cast(image, tf.float32)
    image /= 255.
    return image, label

scaled_train_and_validation_data = mnist_train.map(scale)

test_data = mnist_test.map(scale)


BUFFER_SIZE = 10000

shuffled_train_and_validation_data = scaled_train_and_validation_data.shuffle(BUFFER_SIZE)

validation_data = shuffled_train_and_validation_data.take(num_validation_samples)
train_data = shuffled_train_and_validation_data.skip(num_validation_samples)


BATCH_SIZE = 100

train_data = train_data.batch(BATCH_SIZE)
validation_data = validation_data.batch(num_validation_samples)
test_data = test_data.batch(num_test_samples)

validation_inputs, validation_targets = next(iter(validation_data))

## Model

### Outline the model

In [3]:
input_size = 784
output_size = 10
hidden_layer_size = 50

model = tf.keras.Sequential([
                            tf.keras.layers.Flatten(input_shape=(28,28,1)),
                            tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
                            tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
                            tf.keras.layers.Dense(output_size, activation='softmax')   
                            ])

### Optimizer and loss function

In [4]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

### Training

In [5]:
NUM_EPOCHS = 5
VALIDATION_STEPS = num_validation_samples

model.fit(train_data, epochs = NUM_EPOCHS, validation_data=(validation_inputs, validation_targets), 
          validation_steps=VALIDATION_STEPS, verbose=2)

Epoch 1/5
540/540 - 30s - loss: 0.3925 - accuracy: 0.8888 - val_loss: 0.2111 - val_accuracy: 0.9423
Epoch 2/5
540/540 - 27s - loss: 0.1828 - accuracy: 0.9471 - val_loss: 0.1600 - val_accuracy: 0.9553
Epoch 3/5
540/540 - 31s - loss: 0.1320 - accuracy: 0.9606 - val_loss: 0.1222 - val_accuracy: 0.9655
Epoch 4/5
540/540 - 27s - loss: 0.1092 - accuracy: 0.9673 - val_loss: 0.1038 - val_accuracy: 0.9728
Epoch 5/5
540/540 - 28s - loss: 0.0927 - accuracy: 0.9724 - val_loss: 0.0966 - val_accuracy: 0.9708


## Test the model

In [6]:
test_loss, test_accuracy = model.evaluate(test_data)

      1/Unknown - 3s 3s/step - loss: 0.1071 - accuracy: 0.9694

In [7]:
print('Test loss: {0:.2f}. Test accuracy: {1:.2f}%'.format(test_loss, test_accuracy*100.))

Test loss: 0.11. Test accuracy: 96.94%
